In [ ]:
from utils.general import info, ok, warning, id2file

### Label Data

In [ ]:
import os
import re
relevant_set = set()
irrelevant_set = set()

# Loading new_data
new_data = [line.split(';') for line in open('new_data.csv').read().splitlines()]
relevant_set = relevant_set.union(set([id_ for id_,label in new_data if label.strip()=='R']))
irrelevant_set = irrelevant_set.union(set([id_ for id_,label in new_data if label.strip()=='I']))

# Loading original data
DP_examples_dirpath = '/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/DP-relevant articles/'

first_data = []
for dirpath, dirnames, filenames in os.walk(DP_examples_dirpath):
    for filename in filenames:
        content = open(os.path.join(dirpath,filename),'r').read()
        ids = re.findall('/docview/([^/]*)/',content)
        relevant_set = relevant_set.union(set(ids))
    
# articles containg DP and Canada from that period, that were not deteted by Serperi
GM_dp_dirpath = '/home/ec2-user/SageMaker/data/GM_DP_and_Canada1945_1967/'

files = os.listdir(GM_dp_dirpath)

irrelevant_set = irrelevant_set.union([file_[:-4] for file_ in files if file_[:-4] not in relevant_set and file_.endswith('.xml')])

not_found=[]
for id_ in list(relevant_set)+list(irrelevant_set):
    if id2file(id_) is None:
        not_found.append(id_)
print(f'Not found: {not_found}')
for id_ in not_found:
    relevant_set = relevant_set.difference(set(not_found))
    irrelevant_set = irrelevant_set.difference(set(not_found))
    
info(f'len(relevant_set)   = {len(relevant_set)}')
info(f'len(irrelevant_set) = {len(irrelevant_set)}')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from utils.models import tokenize
from utils.tdmstudio import TDMStudio
import spacy
import string
nlp = spacy.load('en_core_web_sm', disable=['textcat', 'parser','ner'])

vectorizer = TfidfVectorizer(
                             input='content',
                             lowercase=True,
                             preprocessor=None,
                             tokenizer=tokenize,
                             analyzer='word',
                             stop_words=list(nlp.Defaults.stop_words),
                             token_pattern=r"(?u)\b\w\w+\b", #selects tokens of 2 or more alphanumeric char (punctuation is completely ignored and treated as token separator)
                             ngram_range=(1,1), #lower and upper boundary of the range of n-values for different n-grams.
                             max_df=1.0, #ignore terms that have a document frequency strictly higher than given threshold
                             min_df=0.001, #ignore terms that have a document frequency strictly lower than given threshold
                             max_features=50000, #build a vocabulary that only considers the top max_features ordered by term frequency acoss the corpus
                             vocabulary=None, #vocabulary is determined from the input documents
                             norm='l2',
                             use_idf=True,
                             )



def remove_punctuation(word):
    return ''.join([char for char in word if not char in string.punctuation+' '])

def tokenize(str_):
    tokens = [word.lemma_.lower() for word in nlp(str_) if not word.is_stop]
    tokens = [word.replace('\n', '') for word in tokens if not word.isnumeric() and len(remove_punctuation(word))!=0]
    return tokens


corpus = [TDMStudio.get_title_and_text(id2file(id_)) for id_ in list(relevant_set)+list(irrelevant_set)]
corpus = [f'{title}. {text}' for title,text in corpus]



In [ ]:
info('Starting fit...')
vectorizer.fit(corpus)
info('Getting vocab...')
vocab = vectorizer.vocabulary_
info('Creatin X...')
X = vectorizer.transform(corpus)

In [ ]:
import numpy as np
y = np.zeros(shape=(X.shape[0],))
y[:len(relevant_set)]=1
y

In [2]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
                         penalty='l2',
                         dual=True, # Prefer dual=False when n_samples>n_features
                         tol=1e-4,  # tolerance
                         C=1,       # Regularization strength. Smaller value specify stronger regularization
                         fit_intercept=True, 
                         intercept_scaling=1,
                         class_weight=None,
                         solver='lbfgs', #
                         n_jobs=3,
                        )